## Klasifikasi Naive Bayes Pada Data Multinomial
Pada percobaan ini, kita akan menggunakan nilai multinomial untuk melakukan klasifikasi dengan Naive Bayes. **Nilai multinomial adalah data yang nilainya didapatkan dari proses menghitung. Sehingga, pada konteks fitur, nilai multinomial fitur berdasarkan proses perhitungan (counting) probabilitas kemunculan fitur tersebut dalam sebuah data.** Contoh klasik fitur multinomial adalah perhitungan jumlah kata pada klasifikasi teks.Pada percobaan ini, kasus klasifikasi teks diberikan untuk mempermudah pemahaman terhadap algoritma Naive Bayes tipe Multinomial.

Kita akan menggunakan data spam.csv yang berisi data teks sms dengan label spam dan ham. Spam adalah sms sampah, sedangkan ham adalah sebaliknya

## Load Data
Pada tahap ini kita akan loading data ke dalam data frame dan melakukan inspeksi sederhana untuk memastikan apakah kita perlu proses pra pengolahan data sebelum melakukan ekstraksi fitur dan permodelan

In [63]:
import numpy as np 
import pandas as pd

df = pd.read_csv('spam.csv', encoding='latin-1') # spesifiksi encoding diperlukan karena data tidak menggunakan UTF-8
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Terdapat 3 kolom yang tidak bermanfaat untuk proses selanjutnya, maka kita perlu membuang kolom tersebut. Selain itu, untuk memudahkan pembacaan data, kita juga akan mengubah nama kolom v1 yang berupa label dan v2 yang berupa teks sms

## Pra Pengolahan data / Pre processing
Beberapa hal yang akan dilakukan pada tahap ini yaitu,
1. Drop kolom yang tidak digunakan
2. Ubah nama kolom v1 (label) dan v2 (teks sms)
3. Inspeksi Data
4. Encode label
5. Memisahkan fitur dengan label

### Drop Kolom

In [64]:
# drop kolom terakhir dengan fungsi iloc
df = df.drop(df.iloc[:,2:], axis=1)

# cek data
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Ubah Nama Kolom V1 dan V2

In [65]:
# Data untuk rename kolom v1 dan v2 
new_cols = {
    'v1': 'Lables',
    'v2': 'SMS'
}

# renama nama kolom v1 dan v2 
df = df.rename(columns=new_cols)

# cek data
df.head()

,Lables,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Inspek data

In [66]:
# cek jumlah data per kelas 
print(df['Lables'].value_counts())
print('\n')

# cek kelengkpana data
print(df.info())
print('\n')

# cek statistik deskriptif
print(df.describe())

ham     4825
spam     747
Name: Lables, dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Lables  5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None


       Lables                     SMS
count    5572                    5572
unique      2                    5169
top       ham  Sorry, I'll call later
freq     4825                      30


### Encode Label 

In [67]:
# data untuk label 
new_label = {
    'spam' : 1,
    'ham' : 0
}

# encode label 
df['Labels'] = df['Lables'].map(new_label)

df.head()

,Lables,SMS,Labels
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


### Memisahkan fitur dengan label

In [68]:
X = df['SMS'].values
y = df['Labels'].values

## Ekstraksi Fitur
Ekstraksi fitur untuk setiap SMS akan menggunakan konsep Bag of Words. Kita dapat menggunakan fungsi CountVectorizer dari scikit-learn. Akan tetapi untuk mencegah leaking information kita akan melakukan split data terlebih dahulu, baru melakukan transformasi terhadap data training dan testing.

**Konsep "Bag of Words" (BoW) adalah teknik dasar dalam pemrosesan teks yang digunakan untuk mewakili dan mengolah teks. Ide dasar di balik BoW adalah menganggap teks sebagai kumpulan kata-kata yang diurutkan secara acak dan mengabaikan struktur dan urutan kata-kata tersebut. Sebaliknya, BoW hanya memperhatikan frekuensi kemunculan kata-kata dalam teks. Ini adalah konsep yang sederhana namun sangat berguna dalam berbagai tugas pemrosesan teks, seperti analisis sentimen, klasifikasi dokumen, dan pemodelan teks.**

In [69]:
from sklearn.model_selection import train_test_split as tts 
from sklearn.feature_extraction.text import CountVectorizer

# split data training dan data testing
X_train, X_test, y_train, y_test = tts(X,y, test_size=0.2, random_state=50)

# inisiasi countVectorizer
bow = CountVectorizer()

# fitting dan transform  X_train dengan countvectorizer
X_train = bow.fit_transform(X_train)

# Transform X_test
# Mengapa hanya transform? Alasan yang sama dengan kasus pada percobaan ke-3
# Kita tidak menginginkan model mengetahui paramter yang digunakan oleh CountVectorizer untuk fitting data X_train
# Sehingga, data testing dapat tetap menjadi data yang asing bagi model nantinya
X_test = bow.transform(X_test)

In [70]:
print(len(bow.get_feature_names()))
print(f'Dimensi data : ', X_train.shape)

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

## Training dan Evaluasi Model 
Kita akan menggunakan algoritma Multinomial Naive Bayes. Fungsi MultinomialNB dari scikit-learn dapat digunakan pada kasus ini.

In [72]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# inisiasi MultinomialNB
mnb = MultinomialNB()

# fit model / training data
mnb.fit(X_train, y_train)

# prediksi dengan data training
y_pred_train = mnb.predict(X_train)

# evaluasi akurasi data training
acc_train = accuracy_score(y_train, y_pred_train) 

# prediksi dengan data training
y_pred_test =mnb.predict(X_test)

# evaluasi akurasi data testing
acc_test = accuracy_score(y_test, y_pred_test)

# print hasil evaluasi 
print('Hasil evaluasi data train : ', acc_train) 
print('Hasil evaluasi data test : ', acc_train) 


Hasil evaluasi data train :  0.9946152120260264
Hasil evaluasi data test :  0.9946152120260264
